In [ ]:
import time
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
from matplotlib import pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else
'cpu')
#读取一张400x500的图像作为样例
d2l.set_figsize()
img = Image.open('./img/cat2.jpg')
d2l.plt.imshow(img)
d2l.plt.show(img)
#定义绘图函数show_images
def show_images(imgs, num_rows, num_cols, scale=2):
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    for i in range(num_rows):
        for j in range(num_cols):
            # d2l.plt.imshow(imgs[i * num_cols])
            # d2l.plt.show(imgs[i * num_cols])
            axes[i][j].imshow(imgs[i * num_cols + j])
            axes[i][j].axes.get_xaxis().set_visible(False)
            axes[i][j].axes.get_yaxis().set_visible(False)
    d2l.plt.imshow(imgs[(num_rows-1)*num_cols])
    d2l.plt.show(imgs[(num_rows-1)*num_cols])
    return axes
    
#定义辅助函数apply。此函数对输入图像img多次运行图像增广方法aug并且展示所有结果
def apply(img, aug, num_rows=2, num_cols=4, scale=1.5):
    Y = [aug(img) for _ in range(num_rows * num_cols)]
    show_images(Y, num_rows, num_cols, scale)
#一半概率图像水平翻转
apply(img, torchvision.transforms.RandomHorizontalFlip())
#一半概率图像垂直翻转
apply(img, torchvision.transforms.RandomVerticalFlip())
#每次随机裁剪出一块面积为原面积10%到100%的区域，且区域宽高之比随机取0.5到2，然后将该区域的宽高分别缩放到200像素
shape_aug = torchvision.transforms.RandomResizedCrop(200, scale=(0.1, 1), ratio=(0.5, 2))
apply(img, shape_aug)
#将图像亮度随机变化为原亮度的50%到150%
apply(img, torchvision.transforms.ColorJitter(brightness=0.5))
#随机变化图像的色调
apply(img, torchvision.transforms.ColorJitter(hue=0.5))
#随机变化图像的对比度
apply(img, torchvision.transforms.ColorJitter(contrast=0.5))
#设置如何随机变化图像的亮度对比度饱和度以及色调
color_aug = torchvision.transforms.ColorJitter(
brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)
apply(img, color_aug)
#叠加多个图像增广方法
augs = torchvision.transforms.Compose([
torchvision.transforms.RandomHorizontalFlip(), color_aug,
shape_aug])
apply(img, augs)